In [2]:
import numpy
import pandas
from keras.models import Sequential
from keras import layers
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.optimizers import Adam

C:\Users\Ayaz\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
seed = 7
numpy.random.seed(seed)


### Loading & splitting data 

In [4]:
dataframe = pandas.read_csv("iris.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]


### Converting Labels

In [5]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
dummy_y = np_utils.to_categorical(encoded_Y)

### Creating baseline model 

In [9]:
def create_baseline():
    model=Sequential()
    model.add(Dense(8, activation='relu',input_shape=(4,)))
    model.add(Dense(3,activation='softmax'))
    model.compile(optimizer=Adam(lr=0.002), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [48]:
estimator = KerasClassifier(build_fn=create_baseline, epochs=200, batch_size=5, verbose=0)

In [49]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [50]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Baseline: 97.33% (3.27%)


### Smaller model 

In [31]:
def create_smaller():
    model=Sequential()
    model.add(Dense(4, activation='relu',input_shape=(4,)))
    model.add(Dense(3,activation='softmax'))
    model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [32]:
estimator = KerasClassifier(build_fn=create_smaller, epochs=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Smaller: 88.67% (15.22%)


### Larger model 

In [55]:
def create_larger():
    model=Sequential()
    model.add(Dense(32, activation='relu',input_shape=(4,)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [56]:
estimator = KerasClassifier(build_fn=create_larger, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=20, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

overfitting_Result: 96.00% (4.42%)


### Model that overfits 

In [29]:
def create_overfitting_model():
    model=Sequential()
    model.add(Dense(32, activation='relu',input_shape=(4,)))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer=Adam(lr=0.08), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [30]:
estimator = KerasClassifier(build_fn=create_overfitting_model, epochs=400, batch_size=10, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("overfitting_Result: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

overfitting_Result: 44.67% (21.72%)


### Model with keras functional API 

In [25]:
import keras

def functional_api():
    inputs=keras.Input(shape=(4,))
    hl_1=layers.Dense(4,activation='relu')(inputs)
    hl_2=layers.Dense(16, activation='relu')(hl_1)
    outputs=layers.Dense(3, activation='softmax')(hl_2)
    model=keras.Model(inputs,outputs)
    model.compile(optimizer=Adam(lr=0.002),loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [26]:
estimator = KerasClassifier(build_fn=functional_api, epochs=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("functional_API_Result: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

functional_API_Result: 96.89% (3.47%)


### Applying kfold cross-validation on model without scikit-learn 

In [6]:
k = 4
num_val_samples = len(X) // k
num_epochs = 100
all_scores = []

In [10]:
for i in range(k):
    print('processing fold #', i)
    val_data = X[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = dummy_y[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = numpy.concatenate([X[:i * num_val_samples],X[(i + 1) * num_val_samples:]],axis=0)
    partial_train_targets = numpy.concatenate([dummy_y[:i * num_val_samples],dummy_y[(i + 1) * num_val_samples:]],axis=0)
model = create_baseline()
model.fit(partial_train_data, partial_train_targets,epochs=num_epochs, batch_size=1, verbose=0)
val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
all_scores.append(val_mae)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


In [11]:
all_scores

[0.8108108108108109]